In [ ]:
print("For preprocessing the caida 2007 Dataset!!!")

In [ ]:
from collections import Counter
import math
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
def calculate_entropy(data_list):
  count = Counter(data_list)
  probabilities = [count[key] / len(data_list) for key in count.keys()]
  entropy = -sum([p * math.log2(p) for p in probabilities])
  return entropy

In [ ]:
def calculate_and_print_statistics(dataset):
  try:
      dataset = dataset

      src_ip_entropy = calculate_entropy(dataset['Source_IP'])
      src_port_entropy = calculate_entropy(dataset['Source_Port'])
      dst_port_entropy = calculate_entropy(dataset['Destination_Port'])
      protocol_entropy = calculate_entropy(dataset['Protocol'])
      total_packets = len(dataset['No.'])
      bandwidth = dataset['Length'].sum()
    #   print(f"Entropy of source IP address (etpSrcIP): {src_ip_entropy}")
    #   print(f"Entropy of source port (etpSrcP): {src_port_entropy}")
    #   print(f"Entropy of destination port (etpDstP): {dst_port_entropy}")
    #   print(f"Entropy of packet protocol (etpProtocol): {protocol_entropy}")
    #   print(f"Total number of packets (totalPacket): {total_packets}")
    #   print(f"Total bandwidth (totalBandwidth): {bandwidth}")
      return (src_ip_entropy, src_port_entropy, dst_port_entropy, protocol_entropy, total_packets,bandwidth)
  except Exception as e:
      print(f"Error calculating statistics: {e}")
      
      

In [ ]:
def save_data_in_two_second_intervals_with_entropy(df, main_df):


    # Define the time interval (every 2 seconds)
    time_interval = pd.Timedelta(seconds=2)

    # Initialize start time as the first timestamp in the DataFrame
    start_time = df['Time'].iloc[0]

    # List to store DataFrames for each interval
    iw = 0
    total = 0
    min = 99999999999999999
    while start_time <= df['Time'].iloc[-1]:
        # Define end time as start time plus time interval
        end_time = start_time + time_interval
        
        
        # Filter data for the current time interval
        interval_data = df[(df['Time'] >= start_time) & (df['Time'] < end_time)]
        vals = calculate_and_print_statistics(interval_data)
        # main_df = main_df + vals
        main_df.loc[len(main_df)] = vals
        # Append interval data to the list of DataFrames
        # interval_dfs.append(interval_data)
        length = len(interval_data)
        total = total + length
        if length < min:
            min = length
        
        # Update start time for the next interval
        start_time = end_time
        print(f"\rInterval {iw+1} completed...")
        iw = iw+1
        print("--------------------------------------")
    print("len of total ", total) 
    print("Min per 2s ", min)
    return main_df

In [ ]:


def main(parquet_file, label):
    # Read Parquet file into a PyArrow Table
    table = pq.read_table(parquet_file)

    # Convert PyArrow Table to pandas DataFrame
    df = table.to_pandas()
    # Convert timestamp from scientific notation to regular floating-point numbers
    df['Time'] = df['Time'].astype(float)

    # Convert 'Time' column to datetime format
    df['Time'] = pd.to_datetime(df['Time'], unit='s')

    # Sort DataFrame by 'Time' column to ensure it's in chronological order
    df = df.sort_values(by='Time')

    main_dfv = ['etpSrcIP','etpSrcP','etpDstP','etpProtocol','totalPacket','bandwidth']
    main_df = pd.DataFrame(columns = main_dfv)
    main_df_filled = pd.DataFrame(columns = main_dfv)

    # Making and saving
    main_df = save_data_in_two_second_intervals_with_entropy(df, main_df)
    main_df['label'] = label
    file_name = parquet_file.replace('.pcap output_dataframe.parquet', '2s unlabeled_unfilled_entropy.csv')
    main_df.to_csv('done/unfilled/' + file_name, index=False)
    
    # Replace NaN values with 0
    df_filled = df.fillna(0)    
    main_df_filled = save_data_in_two_second_intervals_with_entropy(df_filled, main_df_filled)
    main_df_filled['label'] = label
    file_name = parquet_file.replace('.pcap output_dataframe.parquet', '2s unlabeled_filled_entropy.csv')
    main_df_filled.to_csv('done/filled/' + file_name, index=False)
    
    print("len of df ", len(df))
    print("len of df filled ",len(df_filled))
    print("len of main df", len(main_df))
    print("len of main df filled", len(main_df_filled)) 


    

In [ ]:

parquet_files_0 = []
parquet_files_1 = []

#labeling to 0
parquet_files_0.append('ddostrace.to-victim.20070804_134936.pcap output_dataframe.parquet')
parquet_files_0.append('ddostrace.to-victim.20070804_135436.pcap output_dataframe.parquet')
parquet_files_0.append('ddostrace.to-victim.20070804_135936.pcap output_dataframe.parquet')
parquet_files_0.append('ddostrace.to-victim.20070804_140436.pcap output_dataframe.parquet')

# ## Doubtfull

# parquet_file = 'ddostrace.to-victim.20070804_140936.pcap output_dataframe.parquet'

# #labeling to 1
parquet_files_1.append('ddostrace.to-victim.20070804_141436.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_141936.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_142436.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_142936.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_143436.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_143936.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_144436.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_144936.pcap output_dataframe.parquet')
parquet_files_1.append('ddostrace.to-victim.20070804_145436.pcap output_dataframe.parquet')




for parquet_file0 in parquet_files_0:
    main(parquet_file0, 0)
print("With 0 completed!!!")
for parquet_file1 in parquet_files_1:
    main(parquet_file1, 1)
print("With 1 completed!!!")
print("Done!!!")
